### 1. Imports

In [3]:
# Imports
import gurobipy as gp
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpStatus, PULP_CBC_CMD, GUROBI
import os
import datetime

### 2. Read data

In [4]:
def read_data(year):
    # read data based on given year
    prices_quarterhours = pd.read_excel(f'ID3_15min_{year}.xlsx', header=0)
    prices_hours = pd.read_excel(f'ID3_60min_{year}.xlsx', header=0)
    prices_id_auctions = pd.read_excel(f'ID_auctions_{year}.xlsx', header=0)
    prices_da = pd.read_excel(f'DA_{year}.xlsx', header=0)

    # set index of both to deliverystart
    prices_quarterhours = prices_quarterhours.set_index('DeliveryStart')
    prices_hours = prices_hours.set_index('DeliveryStart')
    prices_id_auctions = prices_id_auctions.set_index('DeliveryStart')
    prices_da = prices_da.set_index('DeliveryStart')

    # set it to datetime, timezone of the dates is Europe/Berlin
    prices_quarterhours.index = pd.to_datetime(prices_quarterhours.index).tz_convert('Europe/Berlin')
    prices_hours.index = pd.to_datetime(prices_hours.index).tz_convert('Europe/Berlin')
    prices_id_auctions.index = pd.to_datetime(prices_id_auctions.index).tz_convert('Europe/Berlin')
    prices_da.index = pd.to_datetime(prices_da.index).tz_convert('Europe/Berlin')

    return prices_quarterhours, prices_hours, prices_id_auctions, prices_da

### 3. Optimization problem

In [5]:
def run_optimization_hours_quarterhours(prices_qh, prices_h, cap, c_rate, roundtrip_eff, cycles_per_year):

    original_prices_h = prices_h.copy()

    # resample prices_h to quarterhours
    prices_h = prices_h.resample('15min').ffill()

    # set index to datetime
    prices_h.index = pd.to_datetime(prices_h.index)

    # Create the 'battery' model
    m_battery = LpProblem('battery', LpMaximize)

    # Create variables using the DataFrame's index
    current_buy_qh = LpVariable.dicts("current_buy_qh", prices_h.index, lowBound=0)
    current_sell_qh = LpVariable.dicts("current_sell_qh", prices_h.index, lowBound=0)

    current_buy_h = LpVariable.dicts("current_buy_h", prices_h.index, lowBound=0)
    current_sell_h = LpVariable.dicts("current_sell_h", prices_h.index, lowBound=0)

    battery_soc = LpVariable.dicts("battery_soc", prices_h.index, lowBound=0)

    # set efficiency as sqrt of roundtrip efficiency
    efficiency = roundtrip_eff**0.5

    # Objective function
    m_battery += lpSum((((current_sell_qh[i]*prices_qh.loc[i,'IndexPrice'] - current_buy_qh[i]*prices_qh.loc[i,'IndexPrice']) * 1.0/4.0) + current_sell_h[i]*prices_h.loc[i,'IndexPrice'] - current_buy_h[i]*prices_h.loc[i,'IndexPrice']) for i in prices_h.index)

    # Constraints
    previous_index = prices_h.index[0]

    previous_index_rounded = previous_index - pd.Timedelta(minutes=previous_index.minute % 60, seconds=previous_index.second, microseconds=previous_index.microsecond)

    for i in prices_h.index[1:]:
        m_battery += battery_soc[i] == battery_soc[previous_index] + current_buy_qh[previous_index]*efficiency*1.0/4 - current_sell_qh[previous_index]*1.0/4/efficiency + current_buy_h[previous_index_rounded]*1.0/4 * efficiency - current_sell_h[previous_index_rounded]*1.0/4/efficiency, f"BatteryBalance_{i}"
        previous_index = i
        previous_index_rounded = previous_index - pd.Timedelta(minutes=previous_index.minute % 60, seconds=previous_index.second, microseconds=previous_index.microsecond)

    m_battery += battery_soc[prices_h.index[0]] == 0, "InitialBatterySOC"

    for i in prices_h.index:
        m_battery += battery_soc[i] <= cap, f"Cap_{i}"

        # round i to the previous full hour (e.g. 12:15 -> 12:00 and 12:00 -> 12:00)
        i_rounded = i - pd.Timedelta(minutes=i.minute % 60, seconds=i.second, microseconds=i.microsecond)
   
        m_battery += current_buy_qh[i] + current_buy_h[i_rounded] <= cap * c_rate, f"BuyRate_{i}"
        m_battery += current_sell_qh[i] + current_sell_h[i_rounded] <= cap * c_rate, f"SellRate_{i}"
        m_battery += current_sell_qh[i]*1.0/4 + current_sell_h[i_rounded] <= battery_soc[i], f"SellVsSOC_{i}"

    m_battery += lpSum(current_buy_qh[i]*efficiency*1.0/4 + current_buy_h[i] * efficiency for i in prices_h.index) <= cycles_per_year * cap, "YearlyCapacity"

    # for all i in prices_qh.index that are not in prices_h.index, current_buy_h[i] and current_sell_h[i] are 0
    for i in prices_h.index:
        if i not in original_prices_h.index:
            m_battery += current_buy_h[i] == 0, f"BuyH_{i}"
            m_battery += current_sell_h[i] == 0, f"SellH_{i}"

    # Solve the problem
    m_battery.solve(PULP_CBC_CMD(msg=0))

    print(f"Status: {LpStatus[m_battery.status]}")
    print(f"Objective value: {m_battery.objective.value()}")

    results = pd.DataFrame(columns=['current_buy_qh', 'current_sell_qh', 'current_buy_h', 'current_sell_h', 'battery_soc'], index=prices_h.index)
    for i in prices_h.index:
        results.loc[i,'current_buy_qh'] = current_buy_qh[i].value()
        results.loc[i,'current_sell_qh'] = current_sell_qh[i].value()
        results.loc[i,'current_buy_h'] = current_buy_h[i].value()
        results.loc[i,'current_sell_h'] = current_sell_h[i].value()
        results.loc[i,'battery_soc'] = battery_soc[i].value()

    return results, m_battery.objective.value()

In [14]:
def run_optimization_quarterhours(prices_qh, cap, c_rate, roundtrip_eff, cycles_per_year):
    
    # Create the 'battery' model
    m_battery = LpProblem('battery', LpMaximize)

    # Create variables using the DataFrame's index
    current_buy_qh = LpVariable.dicts("current_buy_qh", prices_qh.index, lowBound=0)
    current_sell_qh = LpVariable.dicts("current_sell_qh", prices_qh.index, lowBound=0)
    battery_soc = LpVariable.dicts("battery_soc", prices_qh.index, lowBound=0)

    # Objective function
    m_battery += lpSum((current_sell_qh[i]*prices_qh.loc[i,'IndexPrice'] - current_buy_qh[i]*prices_qh.loc[i,'IndexPrice']) * 1.0/4.0 for i in prices_qh.index)

    # Constraints
    previous_index = prices_qh.index[0]

    efficiency = roundtrip_eff**0.5


    for i in prices_qh.index[1:]:
        m_battery += battery_soc[i] == battery_soc[previous_index] + current_buy_qh[previous_index]*efficiency*1.0/4 - current_sell_qh[previous_index]*1.0/4/efficiency, f"BatteryBalance_{i}"
        previous_index = i

    m_battery += battery_soc[prices_qh.index[0]] == 0, "InitialBatterySOC"

    for i in prices_qh.index:
        m_battery += battery_soc[i] <= cap, f"Cap_{i}"
        m_battery += current_buy_qh[i] <= cap * c_rate, f"BuyRate_{i}"
        m_battery += current_sell_qh[i] <= cap * c_rate, f"SellRate_{i}"
        m_battery += current_sell_qh[i]*1.0/4/efficiency <= battery_soc[i], f"SellVsSOC_{i}"

    # set efficiency as sqrt of roundtrip efficiency
 
    m_battery += lpSum(current_buy_qh[i]*efficiency*1.0/4 for i in prices_qh.index) <= cycles_per_year * cap, "YearlyCapacity"

    # Solve the problem
    m_battery.solve(PULP_CBC_CMD(msg=0))

    print(f"Status: {LpStatus[m_battery.status]}")
    print(f"Objective value: {m_battery.objective.value()}")

    results = pd.DataFrame(columns=['current_buy_qh', 'current_sell_qh', 'battery_soc'], index=prices_qh.index)
    for i in prices_qh.index:
        results.loc[i,'current_buy_qh'] = current_buy_qh[i].value()
        results.loc[i,'current_sell_qh'] = current_sell_qh[i].value()
        results.loc[i, 'profit'] = (current_sell_qh[i].value()*prices_qh.loc[i,'IndexPrice'] - current_buy_qh[i].value()*prices_qh.loc[i,'IndexPrice']) * 1.0/4.0
        results.loc[i,'battery_soc'] = battery_soc[i].value()

    return results, m_battery.objective.value()


In [15]:
def run_optimization_hours(prices_h, cap, c_rate, roundtrip_eff, cycles_per_year):

    # Create the 'battery' model
    m_battery = LpProblem('battery', LpMaximize)

    # Create variables using the DataFrame's index
    current_buy_h = LpVariable.dicts("current_buy_h", prices_h.index, lowBound=0)
    current_sell_h = LpVariable.dicts("current_sell_h", prices_h.index, lowBound=0)
    battery_soc = LpVariable.dicts("battery_soc", prices_h.index, lowBound=0)

    # Objective function
    m_battery += lpSum((current_sell_h[i]*prices_h.loc[i,'IndexPrice'] - current_buy_h[i]*prices_h.loc[i,'IndexPrice']) for i in prices_h.index)

    # Constraints
    previous_index = prices_h.index[0]

    efficiency = roundtrip_eff**0.5

    for i in prices_h.index[1:]:
        m_battery += battery_soc[i] == battery_soc[previous_index] + current_buy_h[previous_index]*efficiency - current_sell_h[previous_index]*1.0/efficiency, f"BatteryBalance_{i}"
        previous_index = i

    m_battery += battery_soc[prices_h.index[0]] == 0, "InitialBatterySOC"

    for i in prices_h.index:
        m_battery += battery_soc[i] <= cap, f"Cap_{i}"
        m_battery += current_buy_h[i] <= cap * c_rate, f"BuyRate_{i}"
        m_battery += current_sell_h[i] <= cap * c_rate, f"SellRate_{i}"
        m_battery += current_sell_h[i]*1.0/efficiency <= battery_soc[i], f"SellVsSOC_{i}"

    m_battery += lpSum(current_buy_h[i]*efficiency for i in prices_h.index) <= cycles_per_year * cap, "YearlyCapacity"

    # Solve the problem
    m_battery.solve(PULP_CBC_CMD(msg=0))

    print(f"Status: {LpStatus[m_battery.status]}")
    print(f"Objective value: {m_battery.objective.value()}")

    results = pd.DataFrame(columns=['current_buy_h', 'current_sell_h', 'battery_soc'], index=prices_h.index)
    for i in prices_h.index:
        results.loc[i,'current_buy_h'] = current_buy_h[i].value()
        results.loc[i,'current_sell_h'] = current_sell_h[i].value()
        results.loc[i, 'profit'] = (current_sell_h[i].value()*prices_h.loc[i,'IndexPrice'] - current_buy_h[i].value()*prices_h.loc[i,'IndexPrice'])
        results.loc[i,'battery_soc'] = battery_soc[i].value()

    return results, m_battery.objective.value()


### 4. Run

#### Yearly evaluation

In [17]:
years = ["2021"]


max_cycles_list = [100, 250, 365, 500, 750]
rto_list = [0.7, 0.75, 0.8, 0.86, 0.9, 0.95, 1.0]
c_rate_list = [0.25, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]

parameters = []

for max_cycles in max_cycles_list:
    parameters.append([max_cycles, 0.86, 0.5])

for rto in rto_list:
    parameters.append([365, rto, 0.5])

for c_rate in c_rate_list:
    parameters.append([365, 0.86, c_rate])

for year in years:

    if not os.path.exists(f'./BatteryEvaluation/yearly/{year}/60'):
        os.makedirs(f'./BatteryEvaluation/yearly/{year}/60', exist_ok=True)

    if not os.path.exists(f'./BatteryEvaluation/yearly/{year}/15'):
        os.makedirs(f'./BatteryEvaluation/yearly/{year}/15', exist_ok=True)

    if not os.path.exists(f'./BatteryEvaluation/yearly/{year}/auctions'):
        os.makedirs(f'./BatteryEvaluation/yearly/{year}/auctions', exist_ok=True)

    
    if not os.path.exists(f'./BatteryEvaluation/yearly/{year}/da'):
        os.makedirs(f'./BatteryEvaluation/yearly/{year}/da', exist_ok=True)

    prices_qh, prices_h, prices_auctions, prices_da = read_data(year)

    revenues_hour = pd.DataFrame(columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])

    for parameter in parameters:
        results, revenue = run_optimization_hours(prices_h, 1, parameter[2], parameter[1], parameter[0])
        results.to_csv(f'./BatteryEvaluation/yearly/{year}/60/results_{year}_{parameter[0]}_mc_{parameter[1]}_rto_{parameter[2]}_cr.csv')
        revenues_hour = pd.concat([revenues_hour, pd.DataFrame([[year, parameter[0], parameter[1], parameter[2], revenue]], columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])])
    
    revenues_hour.to_excel(f'./BatteryEvaluation/yearly/{year}/60/revenues.xlsx')

    revenues_qh = pd.DataFrame(columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])

    for parameter in parameters:
        
        results, revenue = run_optimization_quarterhours(prices_qh, 1, parameter[2], parameter[1], parameter[0])
        results.to_csv(f'./BatteryEvaluation/yearly/{year}/15/results_{year}_{parameter[0]}_mc_{parameter[1]}_rto_{parameter[2]}_cr.csv')
        revenues_qh = pd.concat([revenues_qh, pd.DataFrame([[year, parameter[0], parameter[1], parameter[2], revenue]], columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])])
    
    revenues_qh.to_excel(f'./BatteryEvaluation/yearly/{year}/15/revenues.xlsx')

    revenues_auctions = pd.DataFrame(columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])

    for parameter in parameters:
        
        results, revenue = run_optimization_quarterhours(prices_auctions, 1, parameter[2], parameter[1], parameter[0])
        results.to_csv(f'./BatteryEvaluation/yearly/{year}/auctions/results_{year}_{parameter[0]}_mc_{parameter[1]}_rto_{parameter[2]}_cr.csv')
        revenues_auctions = pd.concat([revenues_auctions, pd.DataFrame([[year, parameter[0], parameter[1], parameter[2], revenue]], columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])])
    
    revenues_auctions.to_excel(f'./BatteryEvaluation/yearly/{year}/auctions/revenues.xlsx')


    revenues_da = pd.DataFrame(columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])

    for parameter in parameters:
        results, revenue = run_optimization_hours(prices_da, 1, parameter[2], parameter[1], parameter[0])
        results.to_csv(f'./BatteryEvaluation/yearly/{year}/da/results_{year}_{parameter[0]}_mc_{parameter[1]}_rto_{parameter[2]}_cr.csv')
        revenues_da = pd.concat([revenues_da, pd.DataFrame([[year, parameter[0], parameter[1], parameter[2], revenue]], columns=['year', 'cycle', 'rto', 'c_rate', 'revenue'])])
    
    revenues_da.to_excel(f'./BatteryEvaluation/yearly/{year}/da/revenues.xlsx')


Status: Optimal
Objective value: 14886.794059416581
Status: Optimal
Objective value: 22133.487284213745
Status: Optimal
Objective value: 24789.183165420185
Status: Optimal
Objective value: 26591.43457276555
Status: Optimal
Objective value: 27174.782690835546
Status: Optimal
Objective value: 17073.058571290047
Status: Optimal
Objective value: 19494.88388082451
Status: Optimal
Objective value: 21920.991914807797
Status: Optimal
Objective value: 24789.183165420185
Status: Optimal
Objective value: 26663.202633018514
Status: Optimal
Objective value: 28943.16261711293
Status: Optimal
Objective value: 31175.98499999998
Status: Optimal
Objective value: 20883.246272584427
Status: Optimal
Objective value: 26016.859692928203
Status: Optimal
Objective value: 26884.12491165966
Status: Optimal
Objective value: 26978.21461113049
Status: Optimal
Objective value: 26978.21461113049
Status: Optimal
Objective value: 26978.21461113049
Status: Optimal
Objective value: 26978.21461113049
Status: Optimal
Objec